### Stage 2. Data preparation.

To prepare data I will use Pandas and Numpy libraries.

In [1]:
# Install pandas library
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

For further analysis, we will need the next datasets: 
1. match.csv
2. players.csv
3. hero_names.csv

Create DataFrames from these datasets

In [3]:
match_df = pd.read_csv("match.csv")
players_df = pd.read_csv("players.csv")
hero_names_df = pd.read_csv("hero_names.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'match.csv'

To analyse the data I can use only the Pandas library and its syntaxis. But one of the conditions of our task is to use not only Python, but also SQL. For this purpose I will create a serverless database using built-in sqlite3 module and subsequently will convert our prepared DataFrames to database tables.

In [ ]:
import sqlite3

def create_connection(db_file: str) -> sqlite3.connect:
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)

##### match_df preparation.
I want to examine the next indicators using only the match_df:
- how often had won a team which had Megacreeps (in percents);
- which percentage of games were won by destroying the only one lane (in percents);

For there purposes, we need to interpret barracks_status column in the appropriate way

(IMPORTANT: see docs about these columns here https://wiki.teamfortress.com/wiki/WebAPI/GetMatchDetails#Barracks%20Status)

Reinforced creeps are creeps with increased amount of health and damage. A team has the reinforced creeps on a particular lane if all the defence towers and all the barracks of the enemy team on this lane were destroyed. You cannot attack tier2 tower before you destroyed tier1 tower and so on. 

Megacreeps are hightly reinforced creeps with huge amount of health and damage. A team has the Megacreeps if all the barracks of the enemy team were destroyed.

In the code below, we will convert integers from  the "_status_" columns to its binary representation and
add appropriate columns which will be show us "Megacreeps"  and "naked" Ancient status.

In [ ]:
# Let's define several service functions

import typing as tp
from enum import Enum


class Lanes(Enum):
    top = 0
    middle = 1
    bottom = 2
    

def int_to_binary(number: int) -> str:
    return format(number, "b")


def get_line_barracks_status(number: int, lane: int) -> int:
    # There are two barracks on the same line, 
    # and they are represented by pairs of bits, starting with the least significant bit
    n = 2
    number = number >> n * lane
    binary_repr = int_to_binary(number=number)
    if int(binary_repr[-n:]) == 0:
        return 0
    return 1

def has_megacreeps(row: pd.DataFrame, enemy_team: str) -> bool:
    return not any([row[f"{lane.name}_lane_status_{enemy_team}"] for lane in Lanes])

In [ ]:
# Now, we can add new columns to match_df

barracks_status = "barracks_status"

for team in ("dire", "radiant"):
    init_col = f"{barracks_status}_{team}"
    for lane in Lanes:
        result_col = f"{lane.name}_lane_status_{team}"
        match_df[result_col] = match_df[init_col].apply(
            lambda number: get_line_barracks_status(number=number, lane=lane.value)
        )
        
    # we don't need old barracks_status col anymore
    del match_df[init_col]
    
# Let's add "megacreeps" columns for convenience
match_df[f"m_creeps_dire"] = match_df.apply(lambda row: has_megacreeps(row=row, enemy_team="radiant"), axis=1)
match_df[f"m_creeps_radiant"] = match_df.apply(lambda row: has_megacreeps(row=row, enemy_team="dire"), axis=1)  

# match_df prepared and we can store it into our database
conn = create_connection("./dota.db")
match_df.to_sql("match", conn, if_exists="replace", index=None)

conn.close()

CREATE TABLE "match" (  
&nbsp;&nbsp;&nbsp;&nbsp;"match_id" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"start_time" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"duration" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"tower_status_radiant" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"tower_status_dire" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"first_blood_time" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"game_mode" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"radiant_win" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"negative_votes" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"positive_votes" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"cluster" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"top_lane_status_dire" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"middle_lane_status_dire" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"bottom_lane_status_dire" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"top_lane_status_radiant" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"middle_lane_status_radiant" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"bottom_lane_status_radiant" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"m_creeps_dire" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"m_creeps_radiant" INTEGER  
)

##### players_df preparation.
I want to examine the next indicators using the players_df and match_df:
- top ten most popular heroes;
- top ten heroes with the hightest winrate;
- top ten heroes with the lowest winrate;
- top ten killers (heroes);
- top ten "suicide bomber" (heroes);
- top ten "gold diggers" (heroes);

Players dataset info:
individual players are identified by account_id but there is an option to play anonymously and roughly one third of the account_id are not available. Anonymous users have the value of 0 for account_id. Contains totals for kills, deaths, denies, etc. Player action counts are available, and are indicated by variable names beginning with unit_order_. Counts for reasons for acquiring or losing gold, and gaining experience, have prefixes gold_, and xp_.

During the preparation on players_df, I will:
- replace "player_slot" (int) column with a "radiant" (int) column with 1/0 values;
- replace None (Text) values in "stuns" column with 0.0, and convert column type to float;
- delete unusing columns, such as unit_ liker columns, etc.

In [ ]:
def is_radiant(number: int) -> int:
    if 0 <= number < 5:
        radiant = 1
    else:
        radiant = 0
    return radiant

players_df["radiant"] = players_df.player_slot.apply(is_radiant)

players_df.loc[(players_df["stuns"] == "None"), "stuns"] = "0.0"
players_df["stuns"] = players_df["stuns"].astype(float)

columns_to_delete = [
    col for col in players_df.columns.values 
    if col.startswith(("unit_", "gold_", "item_", "xp_")) and col != "gold_per_min"
]

for col in columns_to_delete:
    del players_df[col]

# players_df prepared and we can store it into our database
conn = create_connection("./dota.db")
players_df.to_sql("players", conn, if_exists="replace", index=None)

# also, let's create hero_names table. Dataset for this table does not need any preparation
hero_names_df.to_sql("heroes", conn, if_exists="replace", index=None)

conn.close()

CREATE TABLE "players" (  
&nbsp;&nbsp;&nbsp;&nbsp;"match_id" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"account_id" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"hero_id" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"player_slot" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"gold" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"gold_per_min" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"kills" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"deaths" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"assists" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"denies" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"last_hits" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"stuns" REAL,
&nbsp;&nbsp;&nbsp;&nbsp;"hero_damage" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"hero_healing" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"tower_damage" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"level" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"leaver_status" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"radiant" INTEGER  
)  
  
CREATE TABLE "heroes" (  
&nbsp;&nbsp;&nbsp;&nbsp;"name" TEXT,  
&nbsp;&nbsp;&nbsp;&nbsp;"hero_id" INTEGER,  
&nbsp;&nbsp;&nbsp;&nbsp;"localized_name" TEXT  
)  

### Stage 3. Metrics Calculation

##### Part 1. Lanes analysis

1. Let's see how often a team wins if they have megacreeps

In [ ]:
m_creeps_winrate_query = """
WITH results AS (
    SELECT SUM(CASE radiant.result WHEN 1 THEN radiant.qty ELSE 0 END) AS radiant_wins
         , SUM(CASE radiant.result WHEN 0 THEN radiant.qty ELSE 0 END) AS radiant_losts
         , SUM(CASE dire.result WHEN 1 THEN dire.qty ELSE 0 END)       AS dire_wins
         , SUM(CASE dire.result WHEN 0 THEN dire.qty ELSE 0 END)       AS dire_losts
    FROM (SELECT radiant_win AS result
               , count(*)    AS qty
          FROM match
          WHERE m_creeps_radiant = 1
          GROUP BY radiant_win) AS radiant
             JOIN
         (SELECT CASE radiant_win WHEN 1 THEN 0 ELSE 1 END AS result
               , count(*)                                  AS qty
          FROM match
          WHERE m_creeps_dire = 1
          GROUP BY radiant_win) AS dire
         ON radiant.result = dire.result)
SELECT (radiant_wins * 100) / (radiant_wins + radiant_losts) AS radiant_wins_percent
     , (dire_wins * 100) / (dire_wins + dire_losts)          AS dire_wins_percent
FROM results
"""

conn = create_connection("./dota.db")
m_creeps_winrate = pd.read_sql_query(m_creeps_winrate_query, conn)
print(m_creeps_winrate)

As we can see, a radiant team as well as a dire team wins with megacreeps in 99% cases.

2. Nomination "Have won middle - have won the game".

Middle lane is the shortest and the most important lane in Dota2.
In this section we will calculate a percent of games where a winner have destroyed 
only the middle lane buildings and have won a game. 

In [ ]:
mid_lane_winrate_query = """
SELECT SUM(res.win_qty) * 100 / SUM(res.total_qty) AS win_rate
FROM (SELECT SUM(radiant_win) AS win_qty
           , COUNT(*)         AS total_qty
      FROM match
      WHERE middle_lane_status_dire = 0
        and bottom_lane_status_dire = 1
        and top_lane_status_dire = 1
      UNION ALL
      SELECT SUM(CASE radiant_win WHEN 1 THEN 0 ELSE 1 END) AS win_qty
           , COUNT(*)                                       AS total_qty
      FROM match
      WHERE middle_lane_status_radiant = 0
        and bottom_lane_status_radiant = 1
        and top_lane_status_radiant = 1) AS res;
"""
mid_lane_winrate = pd.read_sql_query(mid_lane_winrate_query, conn)
print(mid_lane_winrate)

Wow! 61% of all the games ended throught the only one middle lane! Quite interesting, isn't it?
Let's check other two lanes by the same way

In [ ]:
# Top lane

top_lane_winrate_query = mid_lane_winrate_query.replace(
    "middle_lane_status_dire = 0", "middle_lane_status_dire = 1"
).replace(
    "top_lane_status_dire = 1", "top_lane_status_dire = 0"
).replace(
    "middle_lane_status_radiant = 0", "middle_lane_status_radiant = 1"
).replace(
    "top_lane_status_radiant = 1", "top_lane_status_radiant = 0"
)
print(top_lane_winrate_query)

top_lane_winrate = pd.read_sql_query(top_lane_winrate_query, conn)

print(top_lane_winrate)

Only 19%. It is a big difference.

In [ ]:
# Bottom lane

bottom_lane_winrate_query = top_lane_winrate_query.replace(
    "top_lane_status_dire = 0", "top_lane_status_dire = 1"
).replace(
    "bottom_lane_status_dire = 1", "bottom_lane_status_dire = 0"
).replace(
    "top_lane_status_radiant = 0", "top_lane_status_radiant = 1"
).replace(
    "bottom_lane_status_radiant = 1", "bottom_lane_status_radiant = 0"
)
print(bottom_lane_winrate_query)

bottom_lane_winrate = pd.read_sql_query(bottom_lane_winrate_query, conn)

print(bottom_lane_winrate)

29%. A bit better, but it is still much less than middle lane result

#### Part 2. Heroes analysis

###### 1. Top ten most popular heroes

In [ ]:
most_popular_query = """
SELECT h.localized_name AS hero_name, COUNT(*) AS pick_qty
FROM players AS p
JOIN heroes AS h
ON p.hero_id = h.hero_id
GROUP BY h.localized_name
ORDER BY pick_qty DESC
LIMIT 10
"""

most_popular = pd.read_sql_query(most_popular_query, conn)
print(most_popular)

###### 2. Top ten heroes with the hightest winrate

In [ ]:
top_winrate_query = """
WITH winners AS (
    SELECT p.hero_id AS hero_id
         , COUNT(*)  AS wins_qty
    FROM players AS p
    LEFT JOIN match AS m
    ON p.match_id = m.match_id
    WHERE (p.radiant = 1 AND m.radiant_win = 1)
       OR (p.radiant = 0 AND m.radiant_win = 0)
    GROUP BY hero_id
)
SELECT picks.hero_name
     , picks.qty                      AS picks_qty
     , w.wins_qty                     AS wins_qty
     , (w.wins_qty * 100) / picks.qty AS win_rate
FROM winners AS w
      JOIN
     (SELECT h.localized_name AS hero_name
           , h.hero_id        AS hero_id
           , COUNT(*)         AS qty
      FROM players AS p
      JOIN heroes AS h
      ON p.hero_id = h.hero_id
      GROUP BY h.hero_id) AS picks
     ON picks.hero_id = w.hero_id
ORDER BY win_rate DESC
LIMIT 10
"""

top_winrate = pd.read_sql_query(top_winrate_query, conn)
print(top_winrate)

It's interesting, that there is the only one hero in this DataFrame 
who also was represented in the previous result (the most popular heroes): 

That hero is Slardar! 

###### 3. Top ten heroes with the lowest winrate

In [ ]:
lowest_winrate_query = top_winrate_query.replace("DESC", "ASC")
lowest_winrate = pd.read_sql_query(lowest_winrate_query, conn)
print(lowest_winrate)

We can see here a quite popular hero - Lina (8255 picks), 
but at the same time she is the 7th from the bottom by the winrate

###### 4. Top ten killers (AVG kills quantity per match)

In [ ]:
top_killers_query = """
SELECT h.localized_name       AS hero_name
     , ROUND(AVG(p.kills), 2) AS kills_per_match
FROM players AS p
         JOIN heroes AS h
              ON p.hero_id = h.hero_id
GROUP BY h.hero_id
ORDER BY kills_per_match DESC
LIMIT 10
"""
top_killers = pd.read_sql_query(top_killers_query, conn)
print(top_killers)

###### 5. Top ten "suicide bombers" (AVG deaths quantity per match)

In [ ]:
top_feeders_query = top_killers_query.replace("kills", "deaths")
top_feeders = pd.read_sql_query(top_feeders_query, conn)
print(top_feeders)

###### 6. Top ten "gold diggers" (AVG gold_per_minute value per match)

In [ ]:
top_gold_diggers_query = top_killers_query.replace("kills", "gold_per_min")
top_gold_diggers = pd.read_sql_query(top_gold_diggers_query, conn)
print(top_gold_diggers)